<a href="https://colab.research.google.com/github/cundeyu154/PL-Repo/blob/main/%E4%BD%9C%E6%A5%AD%E5%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install gspread

from google.colab import auth
import gspread

auth.authenticate_user()

In [19]:
import gspread

SERVICE_ACCOUNT_FILE = 'gen-lang-client.json'

SPREADSHEET_ID = '18sTbi8c3XVrEujYYB_EtLa8rm4cbJt80ccKZR-kNQSI'

gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)

spreadsheet = gc.open_by_key(SPREADSHEET_ID)

worksheet = spreadsheet.worksheet('Sheet1')

data = worksheet.get_all_records()

print("✅ 成功讀取的課程資料：")
print(data)

✅ 成功讀取的課程資料：
[{'課程主題 ': '機器學習基礎', '日期 ': '2025/11/18', '關鍵字/要求 ': '記得安裝 Python 函式庫', '提醒風格 ': '嚴肅'}, {'課程主題 ': '專題報告技巧', '日期 ': '2025/11/20', '關鍵字/要求 ': '簡報要包含流程圖', '提醒風格 ': '幽默'}, {'課程主題 ': '資料庫設計', '日期 ': '2025/11/25', '關鍵字/要求 ': '複習 SQL 語法', '提醒風格 ': '鼓勵'}]


In [20]:
!pip install -q google-genai

import os
from google import genai
from google.colab import userdata
API_KEY = userdata.get('GEMINI_API_KEY')
os.environ['GEMINI_API_KEY'] = API_KEY
try:
    client = genai.Client(api_key=API_KEY)
    print("✅ Gemini 客戶端初始化成功。")
except Exception as e:
    print(f"❌ 初始化失敗: {e}")

✅ Gemini 客戶端初始化成功。


In [21]:
generated_reminders = []

for item in data:
    主題 = item['課程主題 '].strip()
    日期 = item['日期 '].strip()
    要求 = item['關鍵字/要求 '].strip()
    風格 = item['提醒風格 '].strip()

    prompt = f"""
    您是一位專業的課程提醒助理。請根據以下課程資訊，生成一句簡短、精悍、且包含表情符號（Emoji）的「行前提醒」訊息。

    - **主題:** {主題}
    - **日期:** {日期}
    - **關鍵要求:** {要求}
    - **提醒風格:** {風格}

    請只輸出提醒訊息本身，不要包含任何開頭語、解釋或其他文字。
    """

    print(f"--- 正在為【{主題}】生成提醒 ---")

    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt,
        )

        reminder_text = response.text.strip()

        item['AI提醒訊息'] = reminder_text
        generated_reminders.append(item)

        print(f"✅ 生成結果: {reminder_text}")

    except Exception as e:
        item['AI提醒訊息'] = f"AI 生成失敗: {e}"
        generated_reminders.append(item)
        print(f"❌ AI 呼叫失敗: {e}")


print("\n--- 所有課程提醒生成完畢 ---")
print(generated_reminders)

--- 正在為【機器學習基礎】生成提醒 ---
✅ 生成結果: 2025/11/18 機器學習基礎：⚠️ 請務必提前安裝 Python 函式庫。這是課程前提！⚙️
--- 正在為【專題報告技巧】生成提醒 ---
✅ 生成結果: 2025/11/20「專題報告技巧」來囉！📝 簡報務必附上「流程圖」，不然觀眾可能在你的邏輯裡迷航，找不到北喔！🧭😂
--- 正在為【資料庫設計】生成提醒 ---
✅ 生成結果: 11/25 資料庫設計課程預備！🚀 複習 SQL 語法，讓學習力MAX！💪✨

--- 所有課程提醒生成完畢 ---
[{'課程主題 ': '機器學習基礎', '日期 ': '2025/11/18', '關鍵字/要求 ': '記得安裝 Python 函式庫', '提醒風格 ': '嚴肅', 'AI提醒訊息': '2025/11/18 機器學習基礎：⚠️ 請務必提前安裝 Python 函式庫。這是課程前提！⚙️'}, {'課程主題 ': '專題報告技巧', '日期 ': '2025/11/20', '關鍵字/要求 ': '簡報要包含流程圖', '提醒風格 ': '幽默', 'AI提醒訊息': '2025/11/20「專題報告技巧」來囉！📝 簡報務必附上「流程圖」，不然觀眾可能在你的邏輯裡迷航，找不到北喔！🧭😂'}, {'課程主題 ': '資料庫設計', '日期 ': '2025/11/25', '關鍵字/要求 ': '複習 SQL 語法', '提醒風格 ': '鼓勵', 'AI提醒訊息': '11/25 資料庫設計課程預備！🚀 複習 SQL 語法，讓學習力MAX！💪✨'}]


In [22]:
import datetime
final_data = generated_reminders
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>我的本週課程提醒摘要</title>
    <meta charset="utf-8">
    <style>
        body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background-color: #f4f4f9; padding: 20px; }}
        .container {{ max-width: 800px; margin: auto; background: white; padding: 30px; border-radius: 12px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1); }}
        h2 {{ color: #4A90E2; border-bottom: 3px solid #4A90E2; padding-bottom: 10px; }}
        .reminder-card {{ background-color: #E6F7FF; border-left: 5px solid #4A90E2; margin-bottom: 15px; padding: 15px; border-radius: 8px; }}
        .course-info {{ color: #777; font-size: 0.9em; margin-bottom: 5px; }}
        .ai-message {{ font-size: 1.1em; font-weight: bold; color: #333; }}
        .footer {{ margin-top: 30px; text-align: center; color: #aaa; font-size: 0.8em; }}
    </style>
</head>
<body>
    <div class="container">
        <h2>📚 我的本週課程提醒摘要</h2>
        <p>生成時間: {datetime.datetime.now().strftime('%Y年%m月%d日 %H:%M:%S')}</p>
"""
for item in final_data:
    主題 = item['課程主題 '].strip()
    日期 = item['日期 '].strip()
    訊息 = item['AI提醒訊息']
    html_content += f"""
        <div class="reminder-card">
            <div class="course-info">
                <strong>課程：</strong> {主題} ( {日期} )
            </div>
            <div class="ai-message">
                {訊息}
            </div>
        </div>
    """
html_content += """
        <div class="footer">
            --- 由 AI (Gemini API) 自動生成 ---
        </div>
    </div>
</body>
</html>
"""
filename = "Weekly_Course_Reminders.html"
with open(filename, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"✅ 提醒摘要 HTML 檔案已生成：【{filename}】")
print("您可以在 Colab 左側的檔案列表 (Files) 中找到並下載此檔案。")
from IPython.display import HTML
HTML(html_content)

✅ 提醒摘要 HTML 檔案已生成：【Weekly_Course_Reminders.html】
您可以在 Colab 左側的檔案列表 (Files) 中找到並下載此檔案。
